### Inicialmente é realizando o load das bibliotecas necessárias para o processo

In [1]:
# core
from json import load as jsload
from warnings import catch_warnings, filterwarnings
from random import shuffle
from time import strftime, mktime
from datetime import datetime
from time import time
from sqlite3 import connect
# Captação dos dados
from tweepy import OAuthHandler, API
# NLP
from spacy import load
# Auxiliar
from emoji import is_emoji
from labels import mapping
# Deep Learning
from tez import Model
from torch import sigmoid, no_grad, LongTensor, sort as torch_sort
from torch.nn import Dropout, Linear, BCEWithLogitsLoss
from transformers import SqueezeBertModel, SqueezeBertTokenizer
from sklearn.metrics import roc_curve, auc
from sklearn import metrics, model_selection, preprocessing
from transformers import AdamW, get_linear_schedule_with_warmup

2022-08-24 09:20:47.605614: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-24 09:20:47.605648: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-08-24 09:20:48.852204: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-24 09:20:48.852234: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-24 09:20:48.852250: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (calixto-VirtualBox): /proc/driver/nvidia/version does not exist


### Criando a conexão com o banco de dados

In [2]:
create_table_tweets = "CREATE TABLE IF NOT EXISTS tweets (tweet_id TEXT PRIMARY KEY, user_id TEXT NOT NULL, sentments TEXT, sent_score INTEGER NOT NULL, favorited INTEGER, retweets INTEGER, created_at INTEGER NOT NULL, text TEXT NOT NULL, hashtags TEXT, mentions TEXT, urls TEXT, type TEXT, is_retweet INTEGER, crypto TEXT, FOREIGN KEY (sentments) REFERENCES sentment (sentment_id));"
create_table_users = "CREATE TABLE IF NOT EXISTS users (user_id TEXT PRIMARY KEY, screen_name TEXT, followers INTEGER);"
create_table_sentment = "CREATE TABLE IF NOT EXISTS sentment (sentment_id INTEGER PRIMARY KEY, sentment_char TEXT);"
create_table_indexes = "CREATE TABLE IF NOT EXISTS indexes (index_id INTEGER PRIMARY KEY, forecast REAL NOT NULL, cryptocurrency TEXT NOT NULL, timestamp INTEGER NOT NULL);"
insert_table_tweets = "INSERT INTO tweets(tweet_id, user_id, sentments, sent_score, favorited, retweets, created_at, text, hashtags, mentions, urls, type, is_retweet, crypto) VALUES ('{}', '{}', '{}', {}, {}, {}, {}, '{}', '{}', '{}', '{}', '{}', '{}', '{}')"
insert_table_users = "INSERT INTO users(user_id, screen_name, followers) VALUES ('{}', '{}', {})"
insert_table_sentment = "INSERT INTO sentment(sentment_id, sentment_char) VALUES ({}, '{}')"
insert_table_indexes = "INSERT INTO indexes(index_id, forecast, cryptocurrency, timestamp) VALUES ({}, {}, '{}', {})"
select_table_users = "SELECT * FROM users WHERE user_id = {}"
select_table_tweets = "SELECT * FROM tweets WHERE tweet_id = {}"
update_table_users = "UPDATE users SET followers = {} WHERE user_id = {}"

try:
    conn = connect(".\cryptobot10.db")
    cursor = conn.cursor()
    
    cursor.execute(create_table_tweets)
    cursor.execute(create_table_users)
    cursor.execute(create_table_sentment)
    cursor.execute(create_table_indexes)
except Exception as e:
    print(e)

### Realizando o load modelo pré treinado spacy

In [3]:
nlp = load("en_core_web_md")

### Coloquei as chaves de acesso a minha conta no Twitter em um arquivo separado :D

In [4]:
with open("key.json") as file:
    keys = jsload(file)

In [5]:
auth = OAuthHandler(
    consumer_key = keys["API key"],
    consumer_secret = keys["API secret key"]
)

auth.set_access_token(
    key = keys["Access token"],
    secret = keys["Access token secret"]
)
api = API(auth)

### Carregando o modelo treinado para identificação das emoções em textos

In [6]:
class config:
    def __init__(self):
        self.val_strategy = "batch"

class EmotionClassifier(Model):
    def __init__(self, num_train_steps, num_classes):
        super().__init__()
        self.bert = SqueezeBertModel.from_pretrained("squeezebert/squeezebert-uncased")
        self.bert_drop = Dropout(0.3)
        self.out = Linear(768, num_classes)
    
    def forward(self, ids, mask, targets= None):
        o_2 = self.bert(ids, attention_mask=mask)["pooler_output"]
        b_o = self.bert_drop(o_2)
        output = self.out(b_o)
        loss = self.loss(output, targets)
        acc = self.monitor_metrics(output, targets)
        return output, loss, acc

In [7]:
model = EmotionClassifier(int(43410 / 32 * 10), len(mapping))
model.load("export/model.bin", device="cpu")

Some weights of the model checkpoint at squeezebert/squeezebert-uncased were not used when initializing SqueezeBertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing SqueezeBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing SqueezeBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Criando a função de identificação dos sentimentos

In [8]:
def score_sentence(text, topn=5):
    final_classification = {}
    tokenizer = SqueezeBertTokenizer.from_pretrained("squeezebert/squeezebert-uncased", do_lower_case=True)
    max_len = 35
    with no_grad():
        inputs = tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=max_len,
            padding="max_length",
            truncation=True
        )
        ids = inputs["input_ids"]
        ids = LongTensor(ids).cpu().unsqueeze(0)
        
        attention_mask = inputs["attention_mask"]
        attention_mask = LongTensor(attention_mask).cpu().unsqueeze(0)
        
        output = model.forward(ids, attention_mask)[0]
        output = sigmoid(output)
        
        probas, indices = torch_sort(output)
        
    probas = probas.cpu().numpy()[0][::-1]
    indices = indices.cpu().numpy()[0][::-1]
    
    for i, p in zip(indices[:topn], probas[:topn]):
        final_classification[mapping[i]] = p
        
    return final_classification

### Criando função para estruturação das predições e tratamento dos textos

In [9]:
### Tabela de pesos para cada sentimento
sent_pesos = {
    "admiration":8,
    "amusement":4,
    "anger":-7,
    "annoyance":-9,
    "approval":7,
    "caring":-8,
    "confusion":-5,
    "curiosity":5,
    "desire":9,
    "disappointment":-10,
    "disapproval":-9,
    "disgust":-5,
    "embarrassment":-6,
    "excitement":10,
    "fear":-8,
    "gratitude":6,
    "grief":-2,
    "joy":1,
    "love":2,
    "nervousness":-4,
    "optimism":10,
    "pride":8,
    "realization":3,
    "relief":4,
    "remorse":-10,
    "sadness":-3,
    "surprise":1,
    "neutral":0
}

In [10]:
def caracters_to_remove(w):
    return any([
                  w.is_bracket
                , w.is_punct
                , w.is_quote
                , w.is_stop
                , w.is_space
                , w.text in ["\n"]
                , not w.i and w.text == "RT"
                , is_emoji(w.text)
                ])

def format_date(d):
    return int(mktime(datetime.strptime(d, "%a %b %d %H:%M:%S +0000 %Y").timetuple()))

def buildStructure(raw_tweet):
    tweet = dict(raw_tweet._json)
    clean_text = " ".join([w.text for w in nlp(tweet["full_text"]) if not caracters_to_remove(w)]).replace("'", "")
    
    sentments = score_sentence(clean_text)
    
    structure = {
        "user_id" : tweet["user"]["id_str"],
        "screen_name" : tweet["user"]["screen_name"],
        "followers" : tweet["user"]["followers_count"],
        "retweet_count" : tweet["retweet_count"],
        "favorited" : tweet["favorited"],
        "created_at": format_date(tweet["created_at"]),
        "id": tweet["id_str"],
        "text": clean_text,
        "hashtags": [h["text"] for h in tweet["entities"]["hashtags"]],
        "user_mentions": [m["screen_name"] for m in tweet["entities"]["user_mentions"]],
        "urls": tweet["entities"]["urls"],
        "type": tweet["metadata"]["result_type"],
        "retweet": "retweeted_status" in tweet,
        "sentments": sentments,
        "sent_score": sum([sent_pesos[k] * v for k, v in sentments.items()])
    }
    
    return structure

### É criada uma função anônima para realizar o cálculo do índice para a predição do valor de cada criptomoeda

In [11]:
prediction = lambda Nf, Nrt, S, Dtc : ((1+(Nf*0.01)+(Nrt*0.1))*(S*0.1))/(1+((Dtc-int(time()))*0.001))

### Execução do loop para captação dos dados no Twitter e cálculo do índice

In [ ]:
step = 1
while True:
    crypto = ["bitcoin", "ethereum", "binance coin"][step % 3]
    search = api.search_tweets(
                                q = crypto, 
                                lang = "en", 
                                count = 200, 
                                result_type = "recent", 
                                tweet_mode = "extended"
            )
    tweets = [buildStructure(t) for t in search]
    for t in tweets:
        ans = cursor.execute(select_table_tweets.format(t["id"]))
        if not len(ans.fetchall()):
            insert_tweets = insert_table_tweets.format(t["id"], t["user_id"], " ".join(t["sentments"].keys()), t["sent_score"], int(t["favorited"]), t["retweet_count"], t["created_at"], t["text"], ", ".join(t["hashtags"]), ", ".join(t["user_mentions"]), ", ".join([u["url"] for u in t["urls"]]), t["type"], int(t["retweet"]), crypto)
            cursor.execute(insert_table_tweets.format(t["id"], t["user_id"], " ".join(t["sentments"].keys()), t["sent_score"], int(t["favorited"]), t["retweet_count"], t["created_at"], t["text"], ", ".join(t["hashtags"]), ", ".join(t["user_mentions"]), ", ".join([u["url"] for u in t["urls"]]), t["type"], int(t["retweet"]), crypto))
        ans = cursor.execute(select_table_users.format(t["user_id"]))
        if len(ans.fetchall()):
            cursor.execute(update_table_users.format(t["followers"], t["user_id"]))
        else:
            insert_user = insert_table_users.format(t["user_id"], t["screen_name"], t["followers"])
            cursor.execute(insert_table_users.format(t["user_id"], t["screen_name"], t["followers"]))
    index = sum([prediction(t["followers"], t["retweet_count"], t["sent_score"], t["created_at"]) for t in tweets])
    cursor.execute(insert_table_indexes.format(step, index, crypto, time()))
    conn.commit()
    print(crypto, index, time())
    step += 1

[W NNPACK.cpp:51] Could not initialize NNPACK! Reason: Unsupported hardware.


ethereum 19.497089862862936 1661343959.3094568
binance coin 3.815445145039569 1661344256.6665404
bitcoin 7.831740516344648 1661344568.8751092
ethereum 27.509773580163124 1661344816.1375194
binance coin 3.1851311758291443 1661345099.894798
bitcoin 3.726351158331289 1661345436.0237699
ethereum 17.90791114165169 1661345733.3232286
binance coin 50.99562707821537 1661345888.3315976
bitcoin -4.017581663251486 1661346195.8877745
ethereum 52.2195877089616 1661346499.0624008
binance coin 2.372690922440257 1661346753.292207
bitcoin 5.18693047709104 1661347008.5874763
ethereum 24.119079212795558 1661347290.6838348
binance coin 1.8361172912932742 1661347616.5214794
bitcoin -12.9472165160452 1661347913.5348313
ethereum 16.96517547968244 1661348218.299688
binance coin 13.4630139403947 1661348440.6792126
bitcoin -9.509223432897864 1661348702.804279
ethereum 27.203689361832843 1661348975.4251952
binance coin -1.0513971715365849 1661349179.167872
bitcoin 5.006115177989847 1661349485.6976182
ethereum 40

In [ ]:
ans.close()

In [ ]:
ans = cursor.execute("SELECT * FROM tweets")
ans.fetchall()

In [ ]:
conn.close()

In [ ]:
ans.close()